# WIKL 문서 처리 파이프라인 테스트

AWS Textract 기반 멀티모달 RAG 문서 처리 시스템의 핵심 기능을 단계별로 테스트합니다.

## 파이프라인 개요
1. **환경 설정**: 백엔드 연결 및 AWS 서비스 초기화
2. **문서 업로드**: 다양한 형식의 문서 처리
3. **텍스트 추출**: AWS Textract 기반 OCR 및 구조 분석
4. **청킹 및 임베딩**: 한국어 최적화 텍스트 분할 및 벡터화
5. **검색 테스트**: 하이브리드 검색 성능 검증

## 1. 환경 설정 및 백엔드 연결

In [1]:
# 기본 라이브러리 및 환경 설정
import os
import json
import requests
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()
BACKEND_URL = os.getenv("BACKEND_URL", "http://localhost:8000")

print(f"🔧 백엔드 URL: {BACKEND_URL}")

🔧 백엔드 URL: http://localhost:8000


In [2]:
# 백엔드 연결 확인
def check_backend_connectivity():
    """백엔드 서비스 연결 상태 확인"""
    try:
        openapi_url = f"{BACKEND_URL.rstrip('/')}/openapi.json"
        resp = requests.get(openapi_url, timeout=5)
        resp.raise_for_status()
        
        data = resp.json()
        title = data.get("info", {}).get("title", "Unknown")
        version = data.get("info", {}).get("version", "Unknown")
        
        print(f"✅ 백엔드 연결 성공: {title} v{version}")
        return True
        
    except Exception as e:
        print(f"❌ 백엔드 연결 실패: {e}")
        return False

BACKEND_AVAILABLE = check_backend_connectivity()

✅ 백엔드 연결 성공: (주)웅진 AWS WIKL API v1.0.0


## 2. AWS Textract 설정

In [3]:
# AWS Textract 클라이언트 초기화
import boto3
from botocore.config import Config
from botocore.exceptions import ClientError

# 백엔드 .env 파일에서 AWS 자격증명 로드
backend_env_path = os.path.join(os.path.dirname(os.getcwd()), 'backend', '.env')
if os.path.exists(backend_env_path):
    from dotenv import load_dotenv
    load_dotenv(backend_env_path)
    print(f"✅ 백엔드 환경설정 로드: {backend_env_path}")
else:
    print(f"⚠️ 백엔드 .env 파일을 찾을 수 없습니다: {backend_env_path}")

# AWS 설정 (백엔드 .env에서 로드)
AWS_REGION = os.getenv("AWS_REGION", "ap-northeast-2")
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")

# S3 설정
AWS_S3_BUCKET = os.getenv("AWS_S3_BUCKET", "wikl-file-bucket-20250910")
TEXTRACT_S3_BUCKET = os.getenv("TEXTRACT_S3_BUCKET", AWS_S3_BUCKET)
TEXTRACT_RESULT_PREFIX = os.getenv("TEXTRACT_RESULT_PREFIX", "textract/results")

# Textract 설정
USE_AWS_TEXTRACT = os.getenv("USE_AWS_TEXTRACT", "true").lower() == "true"
TEXTRACT_MODE = os.getenv("TEXTRACT_MODE", "layout")
TEXTRACT_FEATURE_TYPES = os.getenv("TEXTRACT_FEATURE_TYPES", "TABLES,FORMS,LAYOUT").split(",")
TEXTRACT_MAX_PAGES = int(os.getenv("TEXTRACT_MAX_PAGES", "50"))
CONFIDENCE_THRESHOLD = float(os.getenv("TEXTRACT_CONFIDENCE_THRESHOLD", "80.0"))
TEXTRACT_USE_ASYNC_THRESHOLD_SIZE = int(os.getenv("TEXTRACT_USE_ASYNC_THRESHOLD_SIZE", "5242880"))  # 5MB
TEXTRACT_JOB_TIMEOUT = int(os.getenv("TEXTRACT_JOB_TIMEOUT", "600"))  # 10분

print(f"🌍 AWS 리전: {AWS_REGION}")
print(f"🔐 AWS 액세스 키: {AWS_ACCESS_KEY_ID[:10]}..." if AWS_ACCESS_KEY_ID else "❌ AWS 액세스 키 없음")
print(f"🪣 S3 버킷: {TEXTRACT_S3_BUCKET}")
print(f"📄 Textract 사용: {USE_AWS_TEXTRACT}")
print(f"🔧 Textract 모드: {TEXTRACT_MODE}")
print(f"📊 지원 기능: {TEXTRACT_FEATURE_TYPES}")
print(f"📄 최대 페이지: {TEXTRACT_MAX_PAGES}")
print(f"🎯 신뢰도 임계값: {CONFIDENCE_THRESHOLD}%")
print(f"⚡ 비동기 처리 임계값: {TEXTRACT_USE_ASYNC_THRESHOLD_SIZE/1024/1024:.1f}MB")

✅ 백엔드 환경설정 로드: /home/admin/wkms-aws/backend/.env
🌍 AWS 리전: ap-northeast-2
🔐 AWS 액세스 키: AKIA3PHZBP...
🪣 S3 버킷: wikl-file-bucket-20250910
📄 Textract 사용: True
🔧 Textract 모드: layout
📊 지원 기능: ['TABLES', 'FORMS', 'LAYOUT']
📄 최대 페이지: 50
🎯 신뢰도 임계값: 80.0%
⚡ 비동기 처리 임계값: 5.0MB


In [4]:
# AWS 자격증명 확인 및 설정
def check_aws_credentials():
    """AWS 자격증명 상태 확인"""
    try:
        # 현재 자격증명 확인
        import boto3
        sts = boto3.client('sts', region_name=AWS_REGION)
        identity = sts.get_caller_identity()
        
        print(f"🔐 현재 AWS 자격증명:")
        print(f"   Account: {identity.get('Account')}")
        print(f"   UserId: {identity.get('UserId')}")
        print(f"   Arn: {identity.get('Arn')}")
        
        return True
        
    except Exception as e:
        print(f"❌ AWS 자격증명 확인 실패: {e}")
        print("\n🔧 해결 방법:")
        print("1. EC2 역할에 Textract 권한 추가")
        print("2. 또는 AWS CLI로 자격증명 설정: aws configure")
        print("3. 또는 환경변수 설정:")
        print("   export AWS_ACCESS_KEY_ID=your_key")
        print("   export AWS_SECRET_ACCESS_KEY=your_secret")
        return False

check_aws_credentials()

🔐 현재 AWS 자격증명:
   Account: 788648786912
   UserId: AIDA3PHZBPPQITWNRIEOY
   Arn: arn:aws:iam::788648786912:user/wj77107791


True

In [5]:
# Textract 클라이언트 생성 및 연결 테스트
def init_textract_client():
    """AWS Textract 클라이언트 초기화"""
    try:
        # 명시적으로 자격증명 사용
        if AWS_ACCESS_KEY_ID and AWS_SECRET_ACCESS_KEY:
            client = boto3.client(
                'textract',
                region_name=AWS_REGION,
                aws_access_key_id=AWS_ACCESS_KEY_ID,
                aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
                config=Config(
                    retries={'max_attempts': 3, 'mode': 'standard'},
                    read_timeout=60
                )
            )
            print(f"✅ 명시적 자격증명으로 Textract 클라이언트 생성")
        else:
            # 기본 자격증명 체인 사용
            client = boto3.client(
                'textract',
                region_name=AWS_REGION,
                config=Config(
                    retries={'max_attempts': 3, 'mode': 'standard'},
                    read_timeout=60
                )
            )
            print(f"⚠️ 기본 자격증명 체인으로 Textract 클라이언트 생성")
        
        # 더 안전한 연결 테스트 (권한이 적은 작업으로 테스트)
        try:
            # 간단한 권한 확인 - 실제 문서 없이 테스트
            client.detect_document_text(
                Document={'Bytes': b'invalid'}  # 의도적으로 잘못된 데이터
            )
        except ClientError as e:
            error_code = e.response['Error']['Code']
            if error_code in ['InvalidParameterException', 'UnsupportedDocumentException']:
                print("✅ AWS Textract 클라이언트 연결 성공 (권한 확인됨)")
                return client
            elif error_code == 'AccessDeniedException':
                print(f"❌ Textract 권한 부족: {e}")
                print("🔧 해결 필요: 사용자에게 Textract 권한 추가 필요")
                return None
            else:
                print(f"⚠️ 예상치 못한 오류: {error_code}")
                return client  # 다른 오류는 클라이언트가 작동할 수 있음을 의미
                
    except Exception as e:
        print(f"❌ Textract 클라이언트 초기화 실패: {e}")
        return None

textract_client = init_textract_client()

✅ 명시적 자격증명으로 Textract 클라이언트 생성
✅ AWS Textract 클라이언트 연결 성공 (권한 확인됨)
✅ AWS Textract 클라이언트 연결 성공 (권한 확인됨)


## 3. 문서 처리 함수

In [6]:
# 문서 복잡도 판별
def detect_document_complexity(file_path):
    """문서 복잡도를 분석하여 적절한 Textract API 선택"""
    try:
        file_size = os.path.getsize(file_path)
        
        # 파일 크기 기준 (5MB 이상은 복잡한 문서로 간주)
        if file_size > 5 * 1024 * 1024:
            return 'complex'
        
        # PDF의 경우 페이지 수 확인
        if file_path.lower().endswith('.pdf'):
            try:
                import fitz
                doc = fitz.open(file_path)
                page_count = len(doc)
                doc.close()
                
                return 'complex' if page_count > 10 else 'simple'
            except:
                pass
        
        return 'simple'
        
    except Exception as e:
        print(f"⚠️ 복잡도 감지 실패: {e}")
        return 'simple'

# 테스트
test_file = "/home/admin/wkms-aws/jupyter_notebook/data/input_docs/test.pdf"
if os.path.exists(test_file):
    complexity = detect_document_complexity(test_file)
    print(f"📄 {test_file}: {complexity} 모드")

📄 /home/admin/wkms-aws/jupyter_notebook/data/input_docs/test.pdf: complex 모드


## 검증/저장 단계 안내

- 이 단계(다음 코드 셀)는 `result` 객체를 검사해 성공/실패 모두에 대해 산출물을 보존합니다.
- 성공 시: 추출 텍스트(.txt), 메타데이터(.json), 요약(.md)을 `jupyter_notebook/data/output_texts/`에 타임스탬프 파일로 저장합니다.
- 실패 시: 오류 내용을 담은 `error-*.json`을 같은 디렉터리에 저장하여 추후 디버깅에 활용합니다.
- 프리뷰 결과인 경우에도 동일하게 저장되며, 파일명/요약에 프리뷰 표시가 포함됩니다.

In [7]:
# 본 셀은 분리되었습니다. 아래 새 셀들을 순서대로 실행하세요.
# 1) 환경/클라이언트/토글 설정
# 2) S3 유틸리티
# 3) 프리뷰(고속) 처리 함수
# 4) 안정 처리 전략 결정 함수
# 5) 동기/비동기 처리 함수
# 6) 콘텐츠 추출/메타데이터 구성 함수
# 7) 메인 실행 (result 생성)
print("이전의 긴 로직은 하위 셀들로 분리되었습니다. 아래 셀들을 사용하세요.")

이전의 긴 로직은 하위 셀들로 분리되었습니다. 아래 셀들을 사용하세요.


## 1) 환경/클라이언트/토글 설정

- backend `.env`를 통해 AWS 자격과 S3 버킷/리전을 불러옵니다.
- Textract 클라이언트를 안전하게 생성합니다.
- 프리뷰/대기 동작을 제어하는 환경변수 토글도 정의합니다.
- 이 셀은 한 번만 실행하면 됩니다.

In [8]:
# 환경/클라이언트/토글 설정
import os
import boto3
from datetime import datetime

# 이미 앞선 셀에서 .env 로드 및 기본 설정이 되었음을 가정합니다.
# 누락 시에도 안전하게 기본값을 채우도록 방어적 코드를 둡니다.

AWS_REGION = globals().get('AWS_REGION') or os.getenv('AWS_REGION') or 'us-east-1'
AWS_ACCESS_KEY_ID = globals().get('AWS_ACCESS_KEY_ID') or os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = globals().get('AWS_SECRET_ACCESS_KEY') or os.getenv('AWS_SECRET_ACCESS_KEY')
TEXTRACT_S3_BUCKET = globals().get('TEXTRACT_S3_BUCKET') or os.getenv('TEXTRACT_S3_BUCKET') or os.getenv('AWS_S3_BUCKET')
TEXTRACT_JOB_TIMEOUT = int(globals().get('TEXTRACT_JOB_TIMEOUT') or os.getenv('TEXTRACT_JOB_TIMEOUT') or 120)

# 토글: 프리뷰/페이지수/비동기 대기 여부
TEXTRACT_FAST_PREVIEW = os.getenv("TEXTRACT_FAST_PREVIEW", "true").lower() == "true"
TEXTRACT_PREVIEW_PAGES = int(os.getenv("TEXTRACT_PREVIEW_PAGES", "2"))
TEXTRACT_ASYNC_WAIT = os.getenv("TEXTRACT_ASYNC_WAIT", "false").lower() == "true"

# Textract 클라이언트 생성
textract_client = globals().get('textract_client')
if textract_client is None:
    textract_client = boto3.client(
        'textract',
        region_name=AWS_REGION,
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY
    )
print(f"Textract 클라이언트 준비 완료 (region={AWS_REGION})")

Textract 클라이언트 준비 완료 (region=ap-northeast-2)


## 2) S3 유틸리티 함수

- 대용량 문서는 S3에 업로드 후 Textract 비동기 API를 사용합니다.
- 업로드 성공 시 오브젝트 키를 반환하고, 실패 시 None을 반환합니다.

In [9]:
# S3 업로드 유틸리티
import os
import boto3
from datetime import datetime

def upload_to_s3(file_path, bucket_name, object_key=None):
    try:
        s3_client = boto3.client(
            's3',
            region_name=AWS_REGION,
            aws_access_key_id=AWS_ACCESS_KEY_ID,
            aws_secret_access_key=AWS_SECRET_ACCESS_KEY
        )
        if not object_key:
            filename = os.path.basename(file_path)
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            object_key = f"textract/input/{timestamp}_{filename}"
        print(f"📤 S3 업로드: s3://{bucket_name}/{object_key}")
        s3_client.upload_file(file_path, bucket_name, object_key)
        print("✅ 업로드 완료")
        return object_key
    except Exception as e:
        print(f"❌ 업로드 실패: {e}")
        return None

## 3) 프리뷰(고속) 처리 함수

- 대용량 문서에서 빠르게 샘플 텍스트를 얻기 위해 첫 N 페이지를 이미지로 렌더링합니다.
- 각 이미지에 대해 `detect_document_text`를 호출하고, 2열 문서 가독성을 높이기 위해 간단한 좌/우 열 재구성을 수행합니다.
- 결과는 `processing_mode='preview'`로 표기되어 이후 단계(저장/검증)에서 식별됩니다.

In [30]:
# 텍스트 추출 함수들 정의 (향상된 버전 - 다중 컬럼, 표/폼 분석, 회전 보정, 동적 줌, 페이지별 레이아웃)
import io
import boto3
from PIL import Image
import cv2
import numpy as np
import fitz  # PyMuPDF

def auto_orient_and_deskew_image(img_bytes):
    """이미지 회전/기울기 자동 보정"""
    try:
        # OpenCV로 이미지 읽기
        nparr = np.frombuffer(img_bytes, np.uint8)
        img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
        if img is None:
            return img_bytes
        
        # 그레이스케일 변환
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # 텍스트 회전 감지 및 보정 (간단한 버전)
        edges = cv2.Canny(gray, 50, 150, apertureSize=3)
        lines = cv2.HoughLines(edges, 1, np.pi/180, threshold=100)
        
        if lines is not None and len(lines) > 0:
            angles = []
            for rho, theta in lines[:10]:
                angle = np.degrees(theta) - 90
                if -45 <= angle <= 45:
                    angles.append(angle)
            
            if angles:
                median_angle = np.median(angles)
                if abs(median_angle) > 0.5:  # 0.5도 이상 기울어진 경우만 보정
                    h, w = gray.shape[:2]
                    center = (w // 2, h // 2)
                    rotation_matrix = cv2.getRotationMatrix2D(center, median_angle, 1.0)
                    img = cv2.warpAffine(img, rotation_matrix, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
        
        # 보정된 이미지를 바이트로 변환
        _, buffer = cv2.imencode('.png', img)
        return buffer.tobytes()
    except Exception as e:
        print(f"   ⚠️ 이미지 보정 실패: {e}")
        return img_bytes

def compute_dynamic_zoom(page_width_pt, page_height_pt):
    """동적 줌 계산 - 한글 OCR에 최적화"""
    try:
        # 페이지의 더 긴 쪽을 기준으로 계산
        long_side_pt = max(page_width_pt, page_height_pt)
        
        # 목표: ~360 DPI 해상도 (1800픽셀을 기준으로)
        if long_side_pt > 0:
            optimal_zoom = 1800 / long_side_pt
            # 최소 2.0, 최대 4.0으로 제한
            zoom = max(2.0, min(optimal_zoom, 4.0))
        else:
            zoom = 3.0  # 기본값
        
        return zoom
    except Exception:
        return 3.0

def reconstruct_layout_by_page(lines, page_num, layout_settings=None):
    """페이지별 레이아웃 재구성"""
    if not lines:
        return lines
    
    # 설정 확인 (파라미터 우선, 없으면 전역 변수 사용)
    if layout_settings is None:
        layout_settings = {
            'enable_page_specific': globals().get('ENABLE_PAGE_SPECIFIC_LAYOUT', True),
            'first_page_layout': globals().get('FIRST_PAGE_LAYOUT', 'full_page')
        }
    
    enable_page_specific = layout_settings.get('enable_page_specific', True)
    first_page_layout = layout_settings.get('first_page_layout', 'full_page')
    
    if not enable_page_specific:
        # 페이지별 처리 비활성화: 기본 정렬
        return sorted(lines, key=lambda l: (round(l['y'], 2), l['x']))
    
    if page_num == 1 and first_page_layout == 'full_page':
        # 첫 페이지: 전체 페이지 상하 순서
        return sorted(lines, key=lambda l: (round(l['y'], 2), l['x']))
    else:
        # 두 번째 페이지부터: 2열 레이아웃
        if len(lines) < 2:
            return lines
        
        # 중심점(cx) 기준으로 2열 분할
        cxs = [l['cx'] for l in lines]
        cxs_sorted = sorted(set(cxs))
        
        if len(cxs_sorted) < 2:
            # 열이 하나뿐인 경우: 일반 정렬
            return sorted(lines, key=lambda l: (l['y'], l['x']))
        
        # 가장 큰 간격을 찾아 분할점 결정
        gaps = []
        for i in range(len(cxs_sorted) - 1):
            gap_size = cxs_sorted[i+1] - cxs_sorted[i]
            gap_center = (cxs_sorted[i] + cxs_sorted[i+1]) / 2
            gaps.append((gap_size, gap_center))
        
        gaps.sort(reverse=True)  # 가장 큰 간격부터
        threshold = gaps[0][1] if gaps else 0.5
        
        # 좌우 분할
        left_lines = [l for l in lines if l['cx'] <= threshold]
        right_lines = [l for l in lines if l['cx'] > threshold]
        
        # 각 열 내에서 상하 정렬
        left_sorted = sorted(left_lines, key=lambda l: (l['y'], l['x']))
        right_sorted = sorted(right_lines, key=lambda l: (l['y'], l['x']))
        
        # 좌열 → 우열 순서로 결합
        return left_sorted + right_sorted

def extract_text_fast_preview(file_path, page_limit=None, layout_settings=None):
    """빠른 미리보기 모드 - 페이지별 레이아웃 지원"""
    print("🚀 빠른 미리보기 모드로 텍스트 추출 중...")
    
    # 레이아웃 설정 준비
    if layout_settings is None:
        layout_settings = {
            'enable_page_specific': globals().get('ENABLE_PAGE_SPECIFIC_LAYOUT', True),
            'first_page_layout': globals().get('FIRST_PAGE_LAYOUT', 'full_page')
        }
    
    try:
        doc = fitz.open(file_path)
        total_pages = len(doc)
        
        if page_limit:
            total_pages = min(total_pages, page_limit)
            print(f"   📄 페이지 제한: {page_limit}페이지")
        
        print(f"   📖 총 {total_pages}페이지 처리")
        print(f"   🎯 페이지별 레이아웃: {layout_settings['enable_page_specific']}")
        print(f"   📋 첫 페이지 모드: {layout_settings['first_page_layout']}")
        
        all_text = ""
        page_results = []
        
        for page_num in range(total_pages):
            page = doc[page_num]
            
            # 동적 줌 계산
            w_pt = float(page.rect.width)
            h_pt = float(page.rect.height)
            zoom = compute_dynamic_zoom(w_pt, h_pt)
            
            # 이미지 렌더링
            pix = page.get_pixmap(matrix=fitz.Matrix(zoom, zoom), alpha=False)
            img_bytes = pix.tobytes("png")
            
            # 이미지 보정
            img_bytes = auto_orient_and_deskew_image(img_bytes)
            
            # Textract OCR
            response = textract_client.detect_document_text(Document={'Bytes': img_bytes})
            
            # 라인 블록 추출
            lines = []
            for block in response.get('Blocks', []):
                if block.get('BlockType') == 'LINE' and block.get('Geometry'):
                    bbox = block['Geometry'].get('BoundingBox', {})
                    text = (block.get('Text') or '').strip()
                    if text:
                        left = bbox.get('Left', 0.0)
                        top = bbox.get('Top', 0.0)
                        width = bbox.get('Width', 0.0)
                        cx = left + width / 2.0
                        lines.append({'x': left, 'y': top, 'cx': cx, 'text': text})
            
            # 페이지별 레이아웃 재구성
            lines_sorted = reconstruct_layout_by_page(lines, page_num + 1, layout_settings)
            
            # 페이지 텍스트 생성
            page_text = "\n".join(line['text'] for line in lines_sorted)
            all_text += f"\n--- 페이지 {page_num + 1} ---\n{page_text}\n"
            
            # 페이지 결과 저장
            layout_type = 'full_page' if (page_num == 0 and layout_settings.get('first_page_layout') == 'full_page') else 'two_column'
            page_results.append({
                'page': page_num + 1,
                'text': page_text,
                'lines_count': len(lines),
                'layout_type': layout_type
            })
            
            print(f"   ✅ 페이지 {page_num + 1}: {len(lines)}라인, {len(page_text)}자, 레이아웃={layout_type}")
        
        doc.close()
        print(f"✅ 빠른 미리보기 완료: 총 {len(all_text)}자 추출")
        
        return {
            'full_text': all_text,
            'page_results': page_results,
            'total_pages': total_pages,
            'extraction_method': 'fast_preview_with_page_layout',
            'layout_settings': layout_settings
        }
        
    except Exception as e:
        print(f"❌ 빠른 미리보기 실패: {e}")
        import traceback
        traceback.print_exc()
        return None

def extract_full_via_images(file_path, layout_settings=None):
    """이미지 변환 후 전체 문서 처리 - 페이지별 레이아웃 지원"""
    print("🔄 PDF를 이미지로 변환 후 전체 문서 처리 중...")
    
    # 레이아웃 설정 준비
    if layout_settings is None:
        layout_settings = {
            'enable_page_specific': globals().get('ENABLE_PAGE_SPECIFIC_LAYOUT', True),
            'first_page_layout': globals().get('FIRST_PAGE_LAYOUT', 'full_page')
        }
    
    try:
        doc = fitz.open(file_path)
        total_pages = len(doc)
        print(f"   📖 총 {total_pages}페이지 처리")
        print(f"   🎯 페이지별 레이아웃: {layout_settings['enable_page_specific']}")
        print(f"   📋 첫 페이지 모드: {layout_settings['first_page_layout']}")
        
        all_text = ""
        page_results = []
        
        for page_num in range(total_pages):
            print(f"   🔍 페이지 {page_num + 1}/{total_pages} 처리 중...")
            page = doc[page_num]
            
            # 동적 줌 계산
            w_pt = float(page.rect.width)
            h_pt = float(page.rect.height)
            zoom = compute_dynamic_zoom(w_pt, h_pt)
            
            # 이미지 렌더링
            pix = page.get_pixmap(matrix=fitz.Matrix(zoom, zoom), alpha=False)
            img_bytes = pix.tobytes("png")
            
            # 이미지 보정
            img_bytes = auto_orient_and_deskew_image(img_bytes)
            
            # Textract OCR
            response = textract_client.detect_document_text(Document={'Bytes': img_bytes})
            
            # 라인 블록 추출
            lines = []
            for block in response.get('Blocks', []):
                if block.get('BlockType') == 'LINE' and block.get('Geometry'):
                    bbox = block['Geometry'].get('BoundingBox', {})
                    text = (block.get('Text') or '').strip()
                    if text:
                        left = bbox.get('Left', 0.0)
                        top = bbox.get('Top', 0.0)
                        width = bbox.get('Width', 0.0)
                        cx = left + width / 2.0
                        lines.append({'x': left, 'y': top, 'cx': cx, 'text': text})
            
            # 페이지별 레이아웃 재구성
            lines_sorted = reconstruct_layout_by_page(lines, page_num + 1, layout_settings)
            
            # 페이지 텍스트 생성
            page_text = "\n".join(line['text'] for line in lines_sorted)
            all_text += f"\n--- 페이지 {page_num + 1} ---\n{page_text}\n"
            
            # 페이지 결과 저장
            layout_type = 'full_page' if (page_num == 0 and layout_settings.get('first_page_layout') == 'full_page') else 'two_column'
            page_results.append({
                'page': page_num + 1,
                'text': page_text,
                'lines_count': len(lines),
                'layout_type': layout_type
            })
        
        doc.close()
        print(f"✅ 전체 문서 처리 완료: 총 {len(all_text)}자 추출")
        
        return {
            'full_text': all_text,
            'page_results': page_results,
            'total_pages': total_pages,
            'extraction_method': 'full_image_with_page_layout',
            'layout_settings': layout_settings
        }
        
    except Exception as e:
        print(f"❌ 전체 문서 처리 실패: {e}")
        import traceback
        traceback.print_exc()
        return None

print("📦 향상된 텍스트 추출 함수들 로드 완료 (페이지별 레이아웃 지원)")

📦 향상된 텍스트 추출 함수들 로드 완료 (페이지별 레이아웃 지원)


## 4) 안정 처리 전략 결정 함수

- 파일 크기와 복잡도(`complexity`)에 따라 동기/비동기, 분석/기본 텍스트 추출을 결정합니다.
- 5MB 초과 → 비동기(S3 기반)
- 1~5MB → 동기 detect_document_text
- 1MB 이하 → 복잡도가 높으면 analyze_document(LAYOUT), 아니면 detect_document_text

In [11]:
# 안정 처리 전략 결정

def _determine_stable_processing_strategy(file_size, complexity):
    MAX_SYNC_SIZE = 5 * 1024 * 1024  # 5MB
    if file_size > MAX_SYNC_SIZE:
        return {
            'name': '대용량 비동기 처리',
            'method': 'async',
            'expected_time': '1-3분',
            'features': ['LAYOUT'] if complexity == 'complex' else [],
            'api_type': 'start_document_analysis' if complexity == 'complex' else 'start_document_text_detection'
        }
    elif file_size > 1024 * 1024:
        return {
            'name': '중간 크기 동기 처리',
            'method': 'sync',
            'expected_time': '10-30초',
            'features': [],
            'api_type': 'detect_document_text'
        }
    else:
        return {
            'name': '소형 파일 분석',
            'method': 'sync',
            'expected_time': '5-15초',
            'features': ['LAYOUT'] if complexity == 'complex' else [],
            'api_type': 'analyze_document' if complexity == 'complex' else 'detect_document_text'
        }

## 5) 동기/비동기 처리 함수

- 전략에 따라 Textract 동기/비동기 API를 호출합니다.
- 비동기는 S3 업로드 → start_document_* → get_document_* 폴링(또는 프리뷰 대체)의 순서로 진행합니다.
- 예외 발생 시 `result = { success: False, error, pipeline_stage }` 형태로 반환합니다.

In [32]:
# 동기/비동기 처리 구현 (PDF 안전 폴백 + 페이지네이션) - 페이지별 레이아웃 지원
import time
from botocore.exceptions import ClientError


def _paginate_all_results(get_func, job_id):
    blocks = []
    next_token = None
    while True:
        if next_token:
            res = get_func(JobId=job_id, NextToken=next_token)
        else:
            res = get_func(JobId=job_id)
        blocks.extend(res.get('Blocks', []))
        next_token = res.get('NextToken')
        if not next_token:
            break
    return { 'Blocks': blocks }


def _process_sync_for_rag(file_path, layout_settings, strategy):
    try:
        with open(file_path, 'rb') as f:
            file_bytes = f.read()
        if strategy['api_type'] == 'analyze_document' and strategy['features']:
            response = textract_client.analyze_document(Document={'Bytes': file_bytes}, FeatureTypes=strategy['features'])
            return response
        else:
            try:
                return textract_client.detect_document_text(Document={'Bytes': file_bytes})
            except ClientError as ce:
                code = ce.response.get('Error', {}).get('Code')
                if code in ('UnsupportedDocumentException', 'InvalidParameterException'):
                    print(f"⚠️ detect_document_text 실패({code}) → analyze_document(LAYOUT) 폴백 시도")
                    try:
                        return textract_client.analyze_document(Document={'Bytes': file_bytes}, FeatureTypes=['LAYOUT'])
                    except Exception as e2:
                        print(f"⚠️ analyze_document 폴백 실패: {e2} → 전체 이미지 기반 폴백")
                        return extract_full_via_images(file_path, layout_settings)
                raise
    except Exception as e:
        return {"_error": str(e), "_stage": "sync_processing"}


def _process_async_for_rag(file_path, layout_settings, strategy):
    try:
        if not TEXTRACT_S3_BUCKET:
            print("⚠️ S3 버킷 미설정 → 전체 이미지 기반 폴백 사용")
            return extract_full_via_images(file_path, layout_settings)
        s3_key = upload_to_s3(file_path, TEXTRACT_S3_BUCKET)
        if not s3_key:
            return {"_error": "S3 업로드 실패", "_stage": "s3_upload"}
        doc_loc = {'S3Object': {'Bucket': TEXTRACT_S3_BUCKET, 'Name': s3_key}}
        if strategy['api_type'] == 'start_document_analysis' and strategy['features']:
            start = textract_client.start_document_analysis(DocumentLocation=doc_loc, FeatureTypes=strategy['features'])
            get_func = textract_client.get_document_analysis
        else:
            start = textract_client.start_document_text_detection(DocumentLocation=doc_loc)
            get_func = textract_client.get_document_text_detection
        job_id = start['JobId']
        if not TEXTRACT_ASYNC_WAIT:
            preview = extract_text_fast_preview(file_path, TEXTRACT_PREVIEW_PAGES, layout_settings)
            if preview and preview.get('full_text'):
                return preview
        waited = 0
        step = 5
        max_wait = min(TEXTRACT_JOB_TIMEOUT, 300)
        while waited < max_wait:
            time.sleep(step)
            waited += step
            res = get_func(JobId=job_id)
            if res.get('JobStatus') == 'SUCCEEDED':
                full = _paginate_all_results(get_func, job_id)
                return full
            if res.get('JobStatus') == 'FAILED':
                return {"_error": res.get('StatusMessage','Unknown'), "_stage": "async_processing"}
        return {"_error": f"처리 시간 초과 ({max_wait}s)", "_stage": "timeout"}
    except Exception as e:
        return {"_error": str(e), "_stage": "async_processing"}

## 6) 콘텐츠 추출/메타데이터 구성 함수

- Textract 응답에서 텍스트 라인을 모아 긴 텍스트로 병합합니다.
- 평균 신뢰도를 포함한 간단한 레이아웃/콘텐츠 통계를 구성해 RAG 메타데이터를 만듭니다.
- 프리뷰 결과(이미 완성된 result dict)도 그대로 통과시킵니다.

In [33]:
def _build_result_from_response(textract_response, extraction_method="textract_api", source_file_path=None, layout_settings=None):
    """Textract 응답에서 결과 구조체 생성 - 페이지별 레이아웃 및 표/폼 파싱 지원"""
    result = {
        'success': False,
        'text': '',
        'metadata': {
            'extraction_method': extraction_method,
            'source_file': source_file_path,
            'blocks_total': 0,
            'blocks_by_type': {},
            'confidence_stats': {},
            'page_count': 0,
            'layout_settings': layout_settings or {}
        },
        'page_results': [],
        'tables': [],
        'forms': []
    }
    
    try:
        # 블록 정보 수집
        blocks = textract_response.get('Blocks', [])
        result['metadata']['blocks_total'] = len(blocks)
        
        # 블록 타입별 카운트
        block_counts = {}
        for block in blocks:
            block_type = block.get('BlockType', 'UNKNOWN')
            block_counts[block_type] = block_counts.get(block_type, 0) + 1
        result['metadata']['blocks_by_type'] = block_counts
        
        # 페이지별 처리
        pages_data = {}
        line_blocks = []
        
        for block in blocks:
            if block.get('BlockType') == 'PAGE':
                page_id = block.get('Id')
                page_num = len(pages_data) + 1
                pages_data[page_id] = {
                    'page_number': page_num,
                    'geometry': block.get('Geometry', {}),
                    'lines': []
                }
            elif block.get('BlockType') == 'LINE':
                line_blocks.append(block)
        
        result['metadata']['page_count'] = len(pages_data)
        
        # 라인 블록을 페이지별로 그룹화
        for line_block in line_blocks:
            relationships = line_block.get('Relationships', [])
            page_id = None
            
            # 페이지 관계 찾기
            for rel in relationships:
                if rel.get('Type') == 'CHILD':
                    # 부모 페이지 찾기 (역추적)
                    for block in blocks:
                        if (block.get('BlockType') == 'PAGE' and 
                            line_block.get('Id') in [child_id for child_rel in block.get('Relationships', []) 
                                                     if child_rel.get('Type') == 'CHILD' 
                                                     for child_id in child_rel.get('Ids', [])]):
                            page_id = block.get('Id')
                            break
                    break
            
            # 페이지를 찾지 못한 경우 첫 번째 페이지로 배정
            if not page_id and pages_data:
                page_id = list(pages_data.keys())[0]
            
            # 라인 정보 추출
            if page_id and page_id in pages_data:
                geometry = line_block.get('Geometry', {})
                bbox = geometry.get('BoundingBox', {})
                text = (line_block.get('Text') or '').strip()
                
                if text and bbox:
                    left = bbox.get('Left', 0.0)
                    top = bbox.get('Top', 0.0)
                    width = bbox.get('Width', 0.0)
                    cx = left + width / 2.0
                    
                    line_info = {
                        'x': left,
                        'y': top,
                        'cx': cx,
                        'text': text,
                        'confidence': line_block.get('Confidence', 0.0)
                    }
                    pages_data[page_id]['lines'].append(line_info)
        
        # 페이지별 텍스트 재구성 (페이지별 레이아웃 적용)
        all_text_parts = []
        page_results = []
        
        for page_id, page_data in pages_data.items():
            page_num = page_data['page_number']
            lines = page_data['lines']
            
            if lines:
                # 페이지별 레이아웃 재구성
                lines_sorted = reconstruct_layout_by_page(lines, page_num, layout_settings)
                
                # 페이지 텍스트 생성
                page_text = "\n".join(line['text'] for line in lines_sorted)
                all_text_parts.append(f"--- 페이지 {page_num} ---\n{page_text}")
                
                # 신뢰도 통계
                confidences = [line['confidence'] for line in lines if line['confidence'] > 0]
                avg_confidence = sum(confidences) / len(confidences) if confidences else 0.0
                
                # 레이아웃 타입 결정
                layout_type = 'full_page' if (page_num == 1 and layout_settings and layout_settings.get('first_page_layout') == 'full_page') else 'two_column'
                
                page_results.append({
                    'page': page_num,
                    'text': page_text,
                    'lines_count': len(lines),
                    'layout_type': layout_type,
                    'avg_confidence': avg_confidence,
                    'char_count': len(page_text)
                })
        
        # 전체 텍스트 결합
        result['text'] = '\n\n'.join(all_text_parts)
        result['page_results'] = page_results
        
        # 신뢰도 통계 계산
        all_confidences = []
        for page_data in pages_data.values():
            for line in page_data['lines']:
                if line['confidence'] > 0:
                    all_confidences.append(line['confidence'])
        
        if all_confidences:
            result['metadata']['confidence_stats'] = {
                'average': sum(all_confidences) / len(all_confidences),
                'min': min(all_confidences),
                'max': max(all_confidences),
                'count': len(all_confidences)
            }
        
        # 표 파싱 (EXPORT_TABLES_FORMS가 True인 경우)
        if globals().get('EXPORT_TABLES_FORMS', False):
            result['tables'] = parse_tables(textract_response)
            result['forms'] = parse_forms(textract_response)
        
        result['success'] = True
        return result
        
    except Exception as e:
        print(f"❌ 결과 구조체 생성 실패: {e}")
        import traceback
        traceback.print_exc()
        return result

def parse_tables(textract_response):
    """표 블록 파싱 (TSV/CSV 형태로 변환)"""
    tables = []
    
    try:
        blocks = textract_response.get('Blocks', [])
        
        # TABLE 블록 찾기
        for block in blocks:
            if block.get('BlockType') == 'TABLE':
                table_data = {
                    'id': block.get('Id'),
                    'confidence': block.get('Confidence', 0.0),
                    'rows': [],
                    'tsv': '',
                    'csv': ''
                }
                
                # 테이블 셀 정보 수집
                relationships = block.get('Relationships', [])
                cell_ids = []
                for rel in relationships:
                    if rel.get('Type') == 'CHILD':
                        cell_ids.extend(rel.get('Ids', []))
                
                # 셀 블록들 찾기
                cells = []
                for cell_id in cell_ids:
                    for b in blocks:
                        if b.get('Id') == cell_id and b.get('BlockType') == 'CELL':
                            cells.append(b)
                
                # 행/열 정보로 정렬
                cells.sort(key=lambda c: (c.get('RowIndex', 0), c.get('ColumnIndex', 0)))
                
                # 행별로 그룹화
                rows = {}
                for cell in cells:
                    row_idx = cell.get('RowIndex', 0)
                    col_idx = cell.get('ColumnIndex', 0)
                    
                    if row_idx not in rows:
                        rows[row_idx] = {}
                    
                    # 셀 텍스트 추출
                    cell_text = ""
                    cell_relationships = cell.get('Relationships', [])
                    for rel in cell_relationships:
                        if rel.get('Type') == 'CHILD':
                            for word_id in rel.get('Ids', []):
                                for b in blocks:
                                    if b.get('Id') == word_id and b.get('BlockType') == 'WORD':
                                        cell_text += (b.get('Text', '') + " ")
                    
                    rows[row_idx][col_idx] = cell_text.strip()
                
                # TSV/CSV 생성
                tsv_rows = []
                csv_rows = []
                for row_idx in sorted(rows.keys()):
                    row_data = rows[row_idx]
                    max_col = max(row_data.keys()) if row_data else 0
                    
                    tsv_row = []
                    csv_row = []
                    for col_idx in range(max_col + 1):
                        cell_text = row_data.get(col_idx, "")
                        tsv_row.append(cell_text)
                        csv_row.append(f'"{cell_text}"' if ',' in cell_text else cell_text)
                    
                    tsv_rows.append('\t'.join(tsv_row))
                    csv_rows.append(','.join(csv_row))
                
                table_data['tsv'] = '\n'.join(tsv_rows)
                table_data['csv'] = '\n'.join(csv_rows)
                table_data['rows'] = list(rows.values())
                
                tables.append(table_data)
    
    except Exception as e:
        print(f"⚠️ 표 파싱 실패: {e}")
    
    return tables

def parse_forms(textract_response):
    """폼 블록 파싱 (키-값 쌍 추출)"""
    forms = []
    
    try:
        blocks = textract_response.get('Blocks', [])
        
        # KEY_VALUE_SET 블록 찾기
        for block in blocks:
            if block.get('BlockType') == 'KEY_VALUE_SET':
                entity_types = block.get('EntityTypes', [])
                
                if 'KEY' in entity_types:
                    # 키 블록 처리
                    key_text = ""
                    value_text = ""
                    
                    # 키 텍스트 추출
                    relationships = block.get('Relationships', [])
                    for rel in relationships:
                        if rel.get('Type') == 'CHILD':
                            for word_id in rel.get('Ids', []):
                                for b in blocks:
                                    if b.get('Id') == word_id and b.get('BlockType') == 'WORD':
                                        key_text += (b.get('Text', '') + " ")
                        elif rel.get('Type') == 'VALUE':
                            # 연결된 값 블록 찾기
                            for value_id in rel.get('Ids', []):
                                for b in blocks:
                                    if b.get('Id') == value_id and b.get('BlockType') == 'KEY_VALUE_SET':
                                        value_relationships = b.get('Relationships', [])
                                        for value_rel in value_relationships:
                                            if value_rel.get('Type') == 'CHILD':
                                                for value_word_id in value_rel.get('Ids', []):
                                                    for word_b in blocks:
                                                        if (word_b.get('Id') == value_word_id and 
                                                            word_b.get('BlockType') == 'WORD'):
                                                            value_text += (word_b.get('Text', '') + " ")
                    
                    if key_text.strip():
                        forms.append({
                            'key': key_text.strip(),
                            'value': value_text.strip(),
                            'confidence': block.get('Confidence', 0.0)
                        })
    
    except Exception as e:
        print(f"⚠️ 폼 파싱 실패: {e}")
    
    return forms

print("📦 향상된 콘텐츠 추출 파이프라인 로드 완료 (페이지별 레이아웃 + 표/폼 지원)")

📦 향상된 콘텐츠 추출 파이프라인 로드 완료 (페이지별 레이아웃 + 표/폼 지원)


In [22]:
# 전체 문서 추출 토글 설정 + 페이지별 레이아웃 활성화
import os
TEXTRACT_FAST_PREVIEW = False
TEXTRACT_ASYNC_WAIT = True
ENABLE_PAGE_SPECIFIC_LAYOUT = True
FIRST_PAGE_LAYOUT = 'full_page'
SAVE_PAGE_RESULTS = True
os.environ['TEXTRACT_PREVIEW_ZOOM'] = os.getenv('TEXTRACT_PREVIEW_ZOOM', '3.0')
print(f"전체 추출 모드 설정 완료 → FAST_PREVIEW={TEXTRACT_FAST_PREVIEW}, ASYNC_WAIT={TEXTRACT_ASYNC_WAIT}, ZOOM={os.environ['TEXTRACT_PREVIEW_ZOOM']}")
print(f"페이지별 레이아웃 설정 완료 → PAGE_SPECIFIC_LAYOUT={ENABLE_PAGE_SPECIFIC_LAYOUT}, FIRST_PAGE_LAYOUT={FIRST_PAGE_LAYOUT}, SAVE_PAGE_RESULTS={SAVE_PAGE_RESULTS}")

전체 추출 모드 설정 완료 → FAST_PREVIEW=False, ASYNC_WAIT=True, ZOOM=3.0
페이지별 레이아웃 설정 완료 → PAGE_SPECIFIC_LAYOUT=True, FIRST_PAGE_LAYOUT=full_page, SAVE_PAGE_RESULTS=True


### 전체 문서 추출 실행 토글 (프리뷰 비활성화 + 비동기 대기)

- 아래 셀을 실행하면 프리뷰를 끄고(`TEXTRACT_FAST_PREVIEW=false`), 비동기 작업 완료까지 기다립니다(`TEXTRACT_ASYNC_WAIT=true`).
- 한국어 OCR 품질을 위해 이미지 경로 사용 시 확대 배율(zoom)을 3.0 이상으로 유지합니다.

## 7) 메인 실행 (result 생성)

- 파일 크기/복잡도/토글에 따라 프리뷰 또는 안정 모드를 선택합니다.
- 항상 `result` 딕셔너리를 설정하여 다음 검증/저장 셀이 동작하도록 합니다.

## 6.5) 실행 대상 파일/복잡도 설정

- 아래 셀에서 `test_file`과 `complexity`를 설정합니다.
- 기본값: `jupyter_notebook/data/input_docs` 폴더의 최신 PDF를 자동 선택, 복잡도는 파일 크기 기준으로 간단 추정합니다.
- 필요 시 경로/복잡도를 직접 지정하고 실행하세요.

## 6.6) 강제 대상 설정 및 토글 (요청 반영)

- 요청하신 파일로 고정: `/home/admin/wkms-aws/jupyter_notebook/data/input_docs/test.pdf`
- 복잡도는 강제로 `complex`로 지정해 레이아웃(2열 등) 대응 경로를 우선합니다.
- 빠른 결과 확인을 위해 프리뷰 우선, 비동기 대기는 비활성화합니다.

In [15]:
# 실행 대상 강제 오버라이드 (요청 문서)
# - 지정 문서: /home/admin/wkms-aws/jupyter_notebook/data/input_docs/test.pdf
# - 복잡도: complex (레이아웃 분석 강조)
# - 비동기 대기 비활성화 + 프리뷰 활성화 (빠른 피드백)
import os

forced_path = "/home/admin/wkms-aws/jupyter_notebook/data/input_docs/test.pdf"
if not os.path.exists(forced_path):
    print(f"오류: 지정된 파일이 없습니다 → {forced_path}")
else:
    test_file = forced_path
    complexity = 'complex'
    # 프리뷰 우선, 비동기 대기는 하지 않음
    os.environ['TEXTRACT_FAST_PREVIEW'] = 'true'
    os.environ['TEXTRACT_ASYNC_WAIT'] = 'false'
    # 프리뷰 페이지 수 (필요시 조정)
    os.environ['TEXTRACT_PREVIEW_PAGES'] = os.environ.get('TEXTRACT_PREVIEW_PAGES', '2')
    print(f"설정 완료 → test_file={test_file}")
    print(f"complexity={complexity}, TEXTRACT_FAST_PREVIEW={os.getenv('TEXTRACT_FAST_PREVIEW')}, TEXTRACT_ASYNC_WAIT={os.getenv('TEXTRACT_ASYNC_WAIT')}, TEXTRACT_PREVIEW_PAGES={os.getenv('TEXTRACT_PREVIEW_PAGES')}")

설정 완료 → test_file=/home/admin/wkms-aws/jupyter_notebook/data/input_docs/test.pdf
complexity=complex, TEXTRACT_FAST_PREVIEW=true, TEXTRACT_ASYNC_WAIT=false, TEXTRACT_PREVIEW_PAGES=2


In [26]:
# 실행 대상 파일/복잡도 설정 (강제 test.pdf + 토글 설정)
import os

# 사용자 지정 대상
forced_path = '/home/admin/wkms-aws/jupyter_notebook/data/input_docs/test.pdf'
if os.path.exists(forced_path):
    test_file = forced_path
    print(f"강제 설정: test_file = {test_file}")
else:
    print(f"경고: 지정된 파일이 없습니다 → {forced_path}")
    # 유지: 기존 자동 탐색 로직을 사용할 수 있도록 test_file 미설정 상태로 둠

# 복잡도 강제 (레이아웃 고려)
complexity = 'complex'
print("강제 설정: complexity = 'complex'")

# 이미지 기반 처리를 위한 토글 설정 (페이지별 레이아웃 인식 활성화)
TEXTRACT_FAST_PREVIEW = False  # 전체 추출 모드
TEXTRACT_PREVIEW_PAGES = int(os.getenv('TEXTRACT_PREVIEW_PAGES', '2'))
TEXTRACT_ASYNC_WAIT = False  # S3 비동기 대신 이미지 기반 처리
print(f"토글: FAST_PREVIEW={TEXTRACT_FAST_PREVIEW}, PREVIEW_PAGES={TEXTRACT_PREVIEW_PAGES}, ASYNC_WAIT={TEXTRACT_ASYNC_WAIT} (이미지 기반 전체 처리)")

강제 설정: test_file = /home/admin/wkms-aws/jupyter_notebook/data/input_docs/test.pdf
강제 설정: complexity = 'complex'
토글: FAST_PREVIEW=False, PREVIEW_PAGES=2, ASYNC_WAIT=False (이미지 기반 전체 처리)


In [ ]:
# 메인 실행: 프리뷰/안정 모드 선택 및 result 생성 (다중페이지 PDF는 전체 우선) - 페이지별 레이아웃 지원
import os, time

print("🚀 RAG 파이프라인 Textract (프리뷰/안정) 테스트")
print("=" * 60)

if 'test_file' in locals() and 'complexity' in locals():
    if os.path.exists(test_file):
        file_size = os.path.getsize(test_file)
        is_pdf = test_file.lower().endswith('.pdf')
        pages = None
        if is_pdf:
            try:
                import fitz
                with fitz.open(test_file) as _d:
                    pages = len(_d)
            except Exception:
                pages = None
        
        # 레이아웃 설정 준비
        layout_settings = {
            'enable_page_specific': globals().get('ENABLE_PAGE_SPECIFIC_LAYOUT', True),
            'first_page_layout': globals().get('FIRST_PAGE_LAYOUT', 'full_page')
        }
        
        print(f"? 파일: {test_file}")
        print(f"📏 크기: {file_size/1024/1024:.2f}MB")
        print(f"📄 페이지: {pages if pages else '알 수 없음'}")
        print(f"🔧 복잡도: {complexity}")
        print(f"🎯 페이지별 레이아웃: {layout_settings['enable_page_specific']}")
        print(f"📋 첫 페이지 모드: {layout_settings['first_page_layout']}")
        print()
        
        start_time = time.time()
        
        # 처리 전략 결정 (프리뷰 강제 또는 정책별)
        if TEXTRACT_FAST_PREVIEW:
            # 프리뷰 모드 강제
            print("🔍 프리뷰 모드 강제 활성화")
            result = extract_text_fast_preview(test_file, TEXTRACT_PREVIEW_PAGES, layout_settings)
            if result and result.get('full_text'):
                result['success'] = True
                result['metadata'] = result.get('metadata', {})
                result['metadata']['processing_mode'] = 'preview'
                result['metadata']['source_file'] = test_file
                result['metadata']['layout_settings'] = layout_settings
            else:
                result = {'success': False, 'error': '프리뷰 추출 실패', 'metadata': {'processing_mode': 'preview_failed'}}
        
        elif pages and pages >= 5:
            # 다중 페이지 PDF는 이미지 기반 전체 처리 우선
            print(f"? 다중 페이지 PDF ({pages}p) → 이미지 기반 전체 처리")
            result = extract_full_via_images(test_file, layout_settings)
            if result and result.get('full_text'):
                result['success'] = True
                result['metadata'] = result.get('metadata', {})
                result['metadata']['processing_mode'] = 'full_image'
                result['metadata']['source_file'] = test_file
                result['metadata']['layout_settings'] = layout_settings
            else:
                result = {'success': False, 'error': '이미지 기반 전체 처리 실패', 'metadata': {'processing_mode': 'full_image_failed'}}
        
        else:
            # 안정 처리 모드 (전략별)
            strat = _determine_stable_processing_strategy(file_size, complexity)
            print(f"📋 전략: {strat['name']} (예상 시간: {strat['expected_time']})")
            
            if strat['method'] == 'async':
                textract_response = _process_async_for_rag(test_file, layout_settings, strat)
            else:
                textract_response = _process_sync_for_rag(test_file, layout_settings, strat)
            
            # 응답 처리
            if textract_response.get('_error'):
                result = {
                    'success': False,
                    'error': textract_response['_error'],
                    'pipeline_stage': textract_response.get('_stage', 'unknown'),
                    'metadata': {'processing_mode': 'stable_failed', 'strategy': strat}
                }
            elif textract_response.get('extraction_method'):
                # 이미 완성된 result (프리뷰 등)
                result = textract_response
                result['metadata']['processing_mode'] = 'stable_complete'
                result['metadata']['strategy'] = strat
                result['metadata']['layout_settings'] = layout_settings
            else:
                # Textract 응답을 result로 변환
                result = _build_result_from_response(
                    textract_response, 
                    extraction_method=strat['name'],
                    source_file_path=test_file,
                    layout_settings=layout_settings
                )
                result['metadata']['processing_mode'] = 'stable_textract'
                result['metadata']['strategy'] = strat
        
        elapsed = time.time() - start_time
        
        # 결과 요약
        if result.get('success'):
            text_len = len(result.get('text', ''))
            print(f"✅ 처리 성공: {text_len:,}자 추출 ({elapsed:.1f}초)")
            print(f"? 처리 모드: {result.get('metadata', {}).get('processing_mode', '알 수 없음')}")
            print(f"🎯 페이지별 레이아웃: {result.get('metadata', {}).get('layout_settings', {}).get('enable_page_specific', 'N/A')}")
            if result.get('page_results'):
                print(f"📄 페이지 결과: {len(result['page_results'])}개")
                for pr in result['page_results'][:3]:  # 처음 3페이지만 표시
                    print(f"   페이지 {pr['page']}: {pr['lines_count']}라인, {pr['char_count']}자, 타입={pr.get('layout_type', 'N/A')}")
        else:
            error_msg = result.get('error', '알 수 없는 오류')
            print(f"❌ 처리 실패: {error_msg}")
            print(f"🚨 단계: {result.get('pipeline_stage', '알 수 없음')}")
        
        print(f"⏱️ 총 소요 시간: {elapsed:.2f}초")
        print("✅ `result` 변수 설정 완료")
    else:
        print(f"❌ 파일을 찾을 수 없습니다: {test_file}")
        result = {'success': False, 'error': 'File not found', 'metadata': {}}
else:
    print("❌ test_file 또는 complexity가 설정되지 않았습니다.")
    result = {'success': False, 'error': 'Missing configuration', 'metadata': {}}

In [28]:
# RAG 파이프라인 처리 결과 검증 및 저장 (실패 케이스 포함) - 빠른 모드 지원 + 표/폼 내보내기 + 페이지별 저장
import json
import os
from datetime import datetime

# 빠른 검증 모드 토글 및 제한 설정
FAST_VALIDATION = os.getenv('FAST_VALIDATION', 'true').lower() == 'true'
MAX_ANALYSIS_CHARS = int(os.getenv('MAX_ANALYSIS_CHARS', '50000'))  # 긴 텍스트 분석 절단
PRINT_SAMPLE_LINES = int(os.getenv('PRINT_SAMPLE_LINES', '0'))     # 출력 줄 수 제한(기본 0: 미출력)
EXPORT_TABLES_FORMS = os.getenv('EXPORT_TABLES_FORMS', 'true').lower() == 'true'
SAVE_PAGE_RESULTS = os.getenv('SAVE_PAGE_RESULTS', 'true').lower() == 'true'

print("🔍 RAG 파이프라인 처리 결과 검증")
print("=" * 50)

# result 변수 존재 확인
if 'result' not in locals():
    print("❌ 'result' 변수가 없습니다.")
    print("💡 메인 실행 셀을 먼저 실행해 주세요.")
else:
    print("✅ 'result' 변수가 존재합니다.")
    
    # 성공 여부 확인
    success = result.get('success', False)
    
    if success:
        print("✅ 처리가 성공적으로 완료되었습니다.")
        
        # 기본 구조 검증 (성공한 경우)
        required_keys = ['success', 'text', 'blocks_count', 'api_used', 'rag_metadata', 'pipeline_stage']
        missing_keys = [key for key in required_keys if key not in result]
        
        if missing_keys:
            print(f"⚠️ 누락된 키: {missing_keys}")
        else:
            print("✅ 모든 필수 키가 존재합니다.")
        
        # 텍스트 품질 검증
        text = result.get('text', '') or ''
        text_for_analysis = text[:MAX_ANALYSIS_CHARS] if FAST_VALIDATION else text

        if len(text_for_analysis) > 0:
            total_len = len(text)
            print(f"✅ 텍스트 추출 성공: {total_len} 문자 (분석 사용: {len(text_for_analysis)} 문자)")
            
            # 메모리/속도 친화적 라인 분석
            lines = text_for_analysis.split('\n')
            non_empty_lines = [line for line in lines if line.strip()]
            print(f"   📊 총 라인 수(분석): {len(lines)}")
            print(f"   📊 유효 라인 수(분석): {len(non_empty_lines)}")
            print(f"   📊 평균 라인 길이(분석): {len(text_for_analysis)/max(len(non_empty_lines), 1):.1f} 문자")

            # 한국어/영어 비율(샘플 기반)
            sample = text_for_analysis
            total_chars = len(sample.replace(' ', '').replace('\n', ''))
            if total_chars > 0:
                korean_chars = sum(1 for c in sample if '\uac00' <= c <= '\ud7a3')
                english_chars = sum(1 for c in sample if c.isalpha() and c.isascii())
                korean_ratio = (korean_chars / total_chars) * 100
                english_ratio = (english_chars / total_chars) * 100
                print(f"   🇰🇷 한국어 비율(샘플): {korean_ratio:.1f}%")
                print(f"   🇺🇸 영어 비율(샘플): {english_ratio:.1f}%")

            if PRINT_SAMPLE_LINES > 0:
                print("\n--- 텍스트 샘플 ---")
                for ln in lines[:PRINT_SAMPLE_LINES]:
                    print(ln[:200])
                print("--- 샘플 끝 ---\n")
        else:
            print("⚠️ 추출된 텍스트가 없습니다.")
        
        # RAG 메타데이터 검증
        rag_metadata = result.get('rag_metadata', {})
        if rag_metadata:
            print("✅ RAG 메타데이터가 존재합니다.")
            content_types = rag_metadata.get('content_types', {})
            layout_info = rag_metadata.get('layout_info', {})
            print(f"   📄 파일 크기: {rag_metadata.get('file_size', 0)/1024/1024:.2f}MB")
            print(f"   🔧 복잡도: {rag_metadata.get('complexity', 'unknown')}")
            print(f"   📝 처리 방법: {rag_metadata.get('processing_method', 'unknown')}")
            print(f"   📊 텍스트 라인: {content_types.get('text_lines', 0)} | 테이블: {content_types.get('tables', 0)} | 폼: {content_types.get('forms', 0)}")
            print(f"   🎯 페이지별 레이아웃: {layout_info.get('page_specific_layout', False)}")
            
            # 페이지별 결과 정보
            page_results = rag_metadata.get('page_results', [])
            if page_results:
                print(f"   📑 페이지별 결과: {len(page_results)}개 페이지")
                for i, page_info in enumerate(page_results[:3]):  # 처음 3페이지만 미리보기
                    print(f"      페이지 {page_info.get('page_number', i+1)}: {page_info.get('layout_type', 'unknown')} 레이아웃, {page_info.get('line_count', 0)} 라인")
        else:
            print("⚠️ RAG 메타데이터가 없습니다.")
        
        # 다음 단계 준비도 (경량 계산)
        readiness_score = 0
        max_score = 4
        if len(text_for_analysis) > 100:
            readiness_score += 1
        if rag_metadata and rag_metadata.get('ready_for_chunking'):
            readiness_score += 1
        if 'non_empty_lines' in locals() and len(non_empty_lines) >= 5:
            readiness_score += 1
        est_chunks = rag_metadata.get('estimated_chunks', 0) if rag_metadata else 0
        if est_chunks >= 3:
            readiness_score += 1
        readiness_percentage = (readiness_score / max_score) * 100
        print(f"\n📈 전체 준비도(경량): {readiness_percentage:.0f}% ({readiness_score}/{max_score})")

        # 저장
        print(f"\n💾 결과 저장 중...")
        try:
            output_dir = "/home/admin/wkms-aws/jupyter_notebook/data/output_texts"
            os.makedirs(output_dir, exist_ok=True)
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            source_filename = os.path.splitext(os.path.basename(test_file))[0] if 'test_file' in locals() else "unknown"
            base_filename = f"{source_filename}_{timestamp}"

            # 전체 텍스트 저장
            text_filename = f"{base_filename}_extracted_text.txt"
            with open(os.path.join(output_dir, text_filename), 'w', encoding='utf-8') as f:
                f.write(text)
            print(f"✅ 전체 텍스트 저장: {text_filename}")

            # 페이지별 텍스트 저장
            page_results = rag_metadata.get('page_results', []) if rag_metadata else []
            if SAVE_PAGE_RESULTS and page_results:
                for page_info in page_results:
                    page_num = page_info.get('page_number', 1)
                    page_text = page_info.get('text', '')
                    page_filename = f"{base_filename}_page{page_num:02d}.txt"
                    with open(os.path.join(output_dir, page_filename), 'w', encoding='utf-8') as f:
                        f.write(f"# 페이지 {page_num}\n")
                        f.write(f"# 레이아웃: {page_info.get('layout_type', 'unknown')}\n")
                        f.write(f"# 라인 수: {page_info.get('line_count', 0)}\n")
                        f.write(f"# 추정 열 수: {page_info.get('estimated_columns', 0)}\n")
                        f.write(f"# 처리 방법: {page_info.get('processing_method', 'unknown')}\n")
                        f.write("\n" + "="*50 + "\n\n")
                        f.write(page_text)
                
                # 페이지별 메타데이터 저장
                pages_meta_filename = f"{base_filename}_pages_metadata.json"
                with open(os.path.join(output_dir, pages_meta_filename), 'w', encoding='utf-8') as f:
                    json.dump(page_results, f, ensure_ascii=False, indent=2)
                print(f"✅ 페이지별 결과 저장: {len(page_results)}개 페이지, 메타데이터: {pages_meta_filename}")

            # 메타데이터 저장 (경량)
            metadata_filename = f"{base_filename}_metadata.json"
            metadata_to_save = {
                "success": result.get('success', False),
                "api_used": result.get('api_used', 'unknown'),
                "blocks_count": result.get('blocks_count', 0),
                "text_length": len(text),
                "pipeline_stage": result.get('pipeline_stage', 'unknown'),
                "rag_metadata": rag_metadata or {},
                "fast_validation": FAST_VALIDATION,
                "analysis_chars": len(text_for_analysis),
                "readiness_percentage": readiness_percentage,
                "page_results_saved": len(page_results) if page_results else 0
            }
            with open(os.path.join(output_dir, metadata_filename), 'w', encoding='utf-8') as f:
                json.dump(metadata_to_save, f, ensure_ascii=False, indent=2)
            print(f"✅ 메타데이터 저장: {metadata_filename}")

            # 표/폼 저장
            if EXPORT_TABLES_FORMS:
                if result.get('tables'):
                    # 테이블을 TSV와 CSV로 각각 저장
                    from pathlib import Path
                    for idx, table in enumerate(result['tables'], 1):
                        # TSV
                        tsv_path = os.path.join(output_dir, f"{base_filename}_table{idx}.tsv")
                        with open(tsv_path, 'w', encoding='utf-8') as f:
                            for row in table:
                                cells = [str(c).replace('\t', ' ').replace('\n', ' ').strip() for c in row]
                                f.write('\t'.join(cells) + '\n')
                        # CSV
                        csv_path = os.path.join(output_dir, f"{base_filename}_table{idx}.csv")
                        with open(csv_path, 'w', encoding='utf-8') as f:
                            for row in table:
                                cells = ['"' + str(c).replace('"', '""') + '"' for c in row]
                                f.write(','.join(cells) + '\n')
                    print(f"✅ 테이블 {len(result['tables'])}개 저장 (TSV/CSV)")
                if result.get('forms'):
                    forms_path = os.path.join(output_dir, f"{base_filename}_forms.json")
                    with open(forms_path, 'w', encoding='utf-8') as f:
                        json.dump(result['forms'], f, ensure_ascii=False, indent=2)
                    print(f"✅ 폼 키-값 저장: {forms_path}")

            # 요약 저장(간단)
            summary_filename = f"{base_filename}_summary.md"
            with open(os.path.join(output_dir, summary_filename), 'w', encoding='utf-8') as f:
                f.write(f"# 결과 요약\n\n")
                f.write(f"- 원본 파일: `{test_file if 'test_file' in locals() else 'Unknown'}`\n")
                f.write(f"- 처리 방법: {result.get('api_used', 'unknown')}\n")
                f.write(f"- 처리 상태: {'✅ 성공' if result.get('success') else '❌ 실패'}\n")
                f.write(f"- 텍스트 길이: {len(text):,} 문자\n")
                if rag_metadata:
                    f.write(f"- 페이지별 레이아웃: {'✅ 활성화' if rag_metadata.get('layout_info',{}).get('page_specific_layout') else '❌ 비활성화'}\n")
                    f.write(f"- 페이지 수: {len(page_results)}개\n")
                    f.write(f"- 표 수: {rag_metadata.get('content_types',{}).get('tables',0)}개\n")
                    f.write(f"- 폼 수: {rag_metadata.get('content_types',{}).get('forms',0)}개\n")
                f.write(f"- 전체 준비도(경량): {readiness_percentage:.0f}%\n")
                
                # 페이지별 정보 추가
                if page_results:
                    f.write(f"\n## 페이지별 정보\n\n")
                    for page_info in page_results:
                        page_num = page_info.get('page_number', 1)
                        layout_type = page_info.get('layout_type', 'unknown')
                        line_count = page_info.get('line_count', 0)
                        cols = page_info.get('estimated_columns', 0)
                        f.write(f"- 페이지 {page_num}: {layout_type} 레이아웃, {line_count}라인, {cols}열\n")
                        
            print(f"✅ 요약 저장: {summary_filename}")

            print(f"\n📂 저장 위치: {output_dir}")
        except Exception as e:
            print(f"❌ 저장 실패: {e}")

    else:
        # 실패 처리: 오류 JSON 저장
        print("❌ 처리가 실패했습니다.")
        error = result.get('error', 'Unknown error')
        pipeline_stage = result.get('pipeline_stage', 'unknown')
        print(f"   오류: {error}")
        print(f"   실패 단계: {pipeline_stage}")
        print(f"\n💾 실패 정보 저장 중...")
        try:
            output_dir = "/home/admin/wkms-aws/jupyter_notebook/data/output_texts"
            os.makedirs(output_dir, exist_ok=True)
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            source_filename = os.path.splitext(os.path.basename(test_file))[0] if 'test_file' in locals() else "unknown"
            error_filename = f"{source_filename}_{timestamp}_error.json"
            error_info = {
                "timestamp": datetime.now().isoformat(),
                "source_file": test_file if 'test_file' in locals() else "Unknown",
                "error": error,
                "pipeline_stage": pipeline_stage
            }
            with open(os.path.join(output_dir, error_filename), 'w', encoding='utf-8') as f:
                json.dump(error_info, f, ensure_ascii=False, indent=2)
            print(f"✅ 오류 정보 저장: {error_filename}")
        except Exception as e:
            print(f"⚠️ 오류 정보 저장 실패: {e}")

print("\n" + "=" * 50)
print("🏁 검증 및 저장 완료")

🔍 RAG 파이프라인 처리 결과 검증
✅ 'result' 변수가 존재합니다.
✅ 처리가 성공적으로 완료되었습니다.
✅ 모든 필수 키가 존재합니다.
✅ 텍스트 추출 성공: 72555 문자 (분석 사용: 50000 문자)
   📊 총 라인 수(분석): 1464
   📊 유효 라인 수(분석): 1446
   📊 평균 라인 길이(분석): 34.6 문자
   🇰🇷 한국어 비율(샘플): 0.0%
   🇺🇸 영어 비율(샘플): 90.7%
✅ RAG 메타데이터가 존재합니다.
   📄 파일 크기: 11.85MB
   🔧 복잡도: complex
   📝 처리 방법: full-image: detect_document_text on 27 page images (zoom=dynamic, page-aware layout)
   📊 텍스트 라인: 1983 | 테이블: 0 | 폼: 0
   🎯 페이지별 레이아웃: False

📈 전체 준비도(경량): 100% (4/4)

💾 결과 저장 중...
✅ 전체 텍스트 저장: test_20250922_162903_extracted_text.txt
✅ 메타데이터 저장: test_20250922_162903_metadata.json
✅ 요약 저장: test_20250922_162903_summary.md

📂 저장 위치: /home/admin/wkms-aws/jupyter_notebook/data/output_texts

🏁 검증 및 저장 완료


In [29]:
# 페이지별 레이아웃 테스트 - 첫 2페이지만 추출해서 결과 확인
import fitz
import os

test_file = '/home/admin/wkms-aws/jupyter_notebook/data/input_docs/test.pdf'
print("📑 페이지별 레이아웃 테스트")
print("=" * 50)

try:
    doc = fitz.open(test_file)
    print(f"📖 총 페이지 수: {len(doc)}")
    
    for page_num in range(min(2, len(doc))):
        print(f"\n🔍 페이지 {page_num + 1} 처리 중...")
        page = doc[page_num]
        
        # 동적 줌 계산
        w_pt = float(page.rect.width)
        h_pt = float(page.rect.height)
        long_pt = max(w_pt, h_pt)
        z = min(max(1800 / long_pt, 2.0), 4.0) if long_pt > 0 else 3.0
        print(f"   페이지 크기: {w_pt:.1f} x {h_pt:.1f} pt, 줌: {z:.2f}")
        
        # 이미지 렌더링 및 OCR
        pix = page.get_pixmap(matrix=fitz.Matrix(z, z), alpha=False)
        img = pix.tobytes("png")
        resp = textract_client.detect_document_text(Document={'Bytes': img})
        
        # 라인 블록 추출
        lines = []
        for block in resp.get('Blocks', []):
            if block.get('BlockType') == 'LINE' and block.get('Geometry'):
                bbox = block['Geometry'].get('BoundingBox', {})
                text = (block.get('Text') or '').strip()
                if text:
                    left = bbox.get('Left', 0.0)
                    top = bbox.get('Top', 0.0)
                    width = bbox.get('Width', 0.0)
                    cx = left + width / 2.0
                    lines.append({'x': left, 'y': top, 'cx': cx, 'text': text})
        
        print(f"   추출된 라인 수: {len(lines)}")
        
        if page_num == 0:
            # 첫 페이지: 상하 순서 (제목, 저자, 초록 순)
            lines_sorted = sorted(lines, key=lambda l: (round(l['y'], 2), l['x']))
            layout_type = "첫 페이지 (상하 순서)"
        else:
            # 두 번째 페이지: 좌우 2열
            if len(lines) >= 2:
                cxs = [l['cx'] for l in lines]
                cxs_sorted = sorted(set(cxs))
                if len(cxs_sorted) >= 2:
                    # 가장 큰 간격을 찾아 분할점 결정
                    gaps = [(cxs_sorted[i+1] - cxs_sorted[i], (cxs_sorted[i] + cxs_sorted[i+1])/2) for i in range(len(cxs_sorted)-1)]
                    gaps.sort(reverse=True)
                    threshold = gaps[0][1] if gaps else 0.5
                    
                    left_lines = [l for l in lines if l['cx'] <= threshold]
                    right_lines = [l for l in lines if l['cx'] > threshold]
                    
                    left_sorted = sorted(left_lines, key=lambda l: (l['y'], l['x']))
                    right_sorted = sorted(right_lines, key=lambda l: (l['y'], l['x']))
                    
                    lines_sorted = left_sorted + right_sorted
                    layout_type = f"두 번째 페이지 (좌우 2열: 좌{len(left_lines)} + 우{len(right_lines)})"
                else:
                    lines_sorted = sorted(lines, key=lambda l: (l['y'], l['x']))
                    layout_type = "두 번째 페이지 (단일 열)"
            else:
                lines_sorted = lines
                layout_type = "두 번째 페이지 (라인 부족)"
        
        print(f"   레이아웃: {layout_type}")
        
        # 결과 텍스트
        page_text = "\n".join(l['text'] for l in lines_sorted)
        print(f"   텍스트 길이: {len(page_text)} 문자")
        print("   처음 200자:")
        print("   " + page_text[:200].replace('\n', ' ')[:200])
        
        # 페이지별 파일 저장
        output_dir = "/home/admin/wkms-aws/jupyter_notebook/data/output_texts"
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        page_filename = f"test_debug_page{page_num+1:02d}_{timestamp}.txt"
        with open(os.path.join(output_dir, page_filename), 'w', encoding='utf-8') as f:
            f.write(f"# 페이지 {page_num + 1} - {layout_type}\n")
            f.write(f"# 라인 수: {len(lines)}, 텍스트 길이: {len(page_text)}\n")
            f.write("# " + "="*60 + "\n\n")
            f.write(page_text)
        print(f"   저장됨: {page_filename}")
    
    doc.close()
    print(f"\n✅ 페이지별 테스트 완료!")
    
except Exception as e:
    print(f"❌ 오류: {e}")
    import traceback
    traceback.print_exc()

📑 페이지별 레이아웃 테스트
📖 총 페이지 수: 27

🔍 페이지 1 처리 중...
   페이지 크기: 538.0 x 742.0 pt, 줌: 2.43
   추출된 라인 수: 41
   레이아웃: 첫 페이지 (상하 순서)
   텍스트 길이: 2622 문자
   처음 200자:
   22 3 Journal of Information Technology Services 2023# 6F, pp.1-27 https://doi.org/10.9716/KITS.2023.2.3.01 of 2/2/21 L HH HT OHE !7 EL Z7/ 2|2| 172 2 II4 TO Ambidextrous Leadership and Innovative Work
   저장됨: test_debug_page01_20250922_163111.txt

🔍 페이지 2 처리 중...
   페이지 크기: 538.0 x 742.0 pt, 줌: 2.43
   추출된 라인 수: 80
   레이아웃: 두 번째 페이지 (좌우 2열: 좌39 + 우41)
   텍스트 길이: 3578 문자
   처음 200자:
   2 Henry Ameyaw Domfeh 1. Introduction The semiconductor industry is a complicated, cyclical sector with a highly dynamic business climate, where crucial market share and billions of dollars are involv
   저장됨: test_debug_page02_20250922_163113.txt

✅ 페이지별 테스트 완료!


In [ ]:
# 빠른 테스트 - 첫 페이지만 처리해서 레이아웃 확인
test_file = '/home/admin/wkms-aws/jupyter_notebook/data/input_docs/test.pdf'

print("🚀 빠른 레이아웃 테스트 (첫 페이지만)")
print("=" * 40)

# 레이아웃 설정
layout_settings = {
    'enable_page_specific': ENABLE_PAGE_SPECIFIC_LAYOUT,
    'first_page_layout': FIRST_PAGE_LAYOUT
}

print(f"🎯 페이지별 레이아웃: {layout_settings['enable_page_specific']}")
print(f"📋 첫 페이지 모드: {layout_settings['first_page_layout']}")

# 빠른 미리보기 (1페이지만)
result = extract_text_fast_preview(test_file, page_limit=1, layout_settings=layout_settings)

if result and result['success']:
    print(f"\n✅ 추출 완료!")
    print(f"📊 총 텍스트: {len(result['full_text'])}자")
    
    if result['page_results']:
        page1 = result['page_results'][0]
        print(f"📄 페이지 1: {page1['lines_count']}라인, 레이아웃={page1['layout_type']}")
        
        # 첫 200자 출력
        text_preview = result['full_text'][:200].replace('\n', ' ')
        print(f"📝 미리보기: {text_preview}...")
        
        # 파일 저장
        output_dir = "/home/admin/wkms-aws/jupyter_notebook/data/output_texts"
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        quick_filename = f"quick_test_{timestamp}.txt"
        with open(os.path.join(output_dir, quick_filename), 'w', encoding='utf-8') as f:
            f.write(f"# 빠른 테스트 결과\n")
            f.write(f"# 설정: {layout_settings}\n")
            f.write("# " + "="*50 + "\n\n")
            f.write(result['full_text'])
        print(f"💾 저장: {quick_filename}")
    
else:
    print("❌ 추출 실패")

## 4. 한국어 청킹 및 임베딩

In [ ]:
# 한국어 텍스트 처리 설정
def setup_korean_nlp():
    """한국어 NLP 도구 초기화"""
    try:
        from kiwipiepy import Kiwi
        import tiktoken
        
        # 형태소 분석기 초기화
        kiwi = Kiwi()
        
        # 토크나이저 초기화
        tokenizer = tiktoken.get_encoding("cl100k_base")
        
        print("✅ 한국어 NLP 도구 초기화 완료")
        return kiwi, tokenizer
        
    except ImportError as e:
        print(f"❌ 한국어 NLP 라이브러리 설치 필요: {e}")
        return None, None

kiwi, tokenizer = setup_korean_nlp()

In [ ]:
# RAG 파이프라인을 위한 한국어 최적화 청킹
def chunk_korean_text_for_rag(text, max_tokens=1200, overlap_ratio=0.1, rag_metadata=None):
    """RAG 파이프라인을 위한 한국어 텍스트 청킹 - 문서 특성 고려"""
    print(f"✂️ RAG 파이프라인 한국어 청킹 시작")
    
    if rag_metadata:
        print(f"📋 문서 메타데이터 기반 최적화:")
        print(f"   파일 크기: {rag_metadata['file_size']/1024/1024:.2f}MB")
        print(f"   복잡도: {rag_metadata['complexity']}")
        print(f"   콘텐츠 구성: 텍스트 {rag_metadata['content_types']['text_lines']}줄")
        
        # 문서 특성에 따른 청킹 전략 조정
        if rag_metadata['complexity'] == 'complex':
            max_tokens = int(max_tokens * 1.2)  # 복잡한 문서는 더 큰 청크
            overlap_ratio = 0.15  # 더 많은 겹침
            print(f"🔧 복잡한 문서 최적화: 청크 크기 {max_tokens}, 겹침 {overlap_ratio*100:.0f}%")
        
        if rag_metadata['content_types']['tables'] > 0:
            print(f"📊 테이블 포함 문서: 구조 보존 청킹 적용")
    
    if not tokenizer:
        print("⚠️ 토크나이저 없음 - 기본 문장 분할 사용")
        # 간단한 문장 기반 분할 (fallback)
        sentences = text.split('.')
        chunks = []
        current_chunk = ""
        
        for sentence in sentences:
            if len(current_chunk + sentence) > max_tokens * 4:  # 대략적인 추정
                if current_chunk:
                    chunks.append({
                        "text": current_chunk.strip(),
                        "tokens_estimated": len(current_chunk.strip()) // 4,
                        "chunk_type": "sentence_based"
                    })
                current_chunk = sentence
            else:
                current_chunk += sentence + "."
        
        if current_chunk:
            chunks.append({
                "text": current_chunk.strip(),
                "tokens_estimated": len(current_chunk.strip()) // 4,
                "chunk_type": "sentence_based"
            })
        
        return chunks
    
    # 토큰 기반 정밀 청킹
    print(f"🔍 토큰 기반 정밀 청킹 (목표: {max_tokens} 토큰, 겹침: {overlap_ratio*100:.0f}%)")
    
    # 텍스트 전처리 - RAG 파이프라인 최적화
    processed_text = text.replace('\n\n', '\n').replace('\r', '')
    sentences = processed_text.split('\n')
    
    chunks = []
    current_chunk = ""
    current_tokens = 0
    chunk_index = 0
    
    for sentence in sentences:
        sentence = sentence.strip()
        if not sentence:
            continue
            
        sentence_tokens = len(tokenizer.encode(sentence))
        
        if current_tokens + sentence_tokens > max_tokens and current_chunk:
            # 현재 청크 완성
            chunk_info = {
                "text": current_chunk.strip(),
                "tokens": current_tokens,
                "chunk_index": chunk_index,
                "chunk_type": "token_based",
                "sentences_count": len([s for s in current_chunk.split('\n') if s.strip()]),
                "overlap_with_next": True if chunk_index > 0 else False
            }
            chunks.append(chunk_info)
            
            # 겹침 처리
            overlap_tokens = int(max_tokens * overlap_ratio)
            if overlap_tokens > 0:
                overlap_sentences = current_chunk.split('\n')[-int(overlap_tokens/50):] # 대략적 계산
                overlap_text = '\n'.join(overlap_sentences)
                current_chunk = overlap_text + "\n" + sentence
                current_tokens = len(tokenizer.encode(current_chunk))
            else:
                current_chunk = sentence
                current_tokens = sentence_tokens
            
            chunk_index += 1
        else:
            current_chunk += "\n" + sentence if current_chunk else sentence
            current_tokens += sentence_tokens
    
    # 마지막 청크 처리
    if current_chunk.strip():
        chunk_info = {
            "text": current_chunk.strip(),
            "tokens": current_tokens,
            "chunk_index": chunk_index,
            "chunk_type": "token_based",
            "sentences_count": len([s for s in current_chunk.split('\n') if s.strip()]),
            "overlap_with_next": False
        }
        chunks.append(chunk_info)
    
    # RAG 파이프라인 품질 검증
    total_tokens = sum(chunk["tokens"] for chunk in chunks)
    avg_tokens = total_tokens / len(chunks) if chunks else 0
    
    print(f"✅ 청킹 완료:")
    print(f"   총 청크 수: {len(chunks)}개")
    print(f"   평균 토큰 수: {avg_tokens:.1f}")
    print(f"   총 토큰 수: {total_tokens}")
    print(f"   청킹 효율성: {(len(chunks) / max(1, len(text)//1000)) * 100:.1f}%")
    
    return chunks

def analyze_chunks_for_rag(chunks):
    """RAG 파이프라인을 위한 청크 분석"""
    print(f"\n📊 RAG 청크 품질 분석:")
    
    if not chunks:
        print("❌ 분석할 청크가 없습니다")
        return
    
    # 청크 크기 분포
    chunk_sizes = [chunk.get("tokens", len(chunk["text"])//4) for chunk in chunks]
    min_size, max_size = min(chunk_sizes), max(chunk_sizes)
    avg_size = sum(chunk_sizes) / len(chunk_sizes)
    
    print(f"   청크 크기 분포: 최소 {min_size}, 최대 {max_size}, 평균 {avg_size:.1f} 토큰")
    
    # 청크 균일성 검사
    size_variance = sum((size - avg_size) ** 2 for size in chunk_sizes) / len(chunk_sizes)
    uniformity_score = max(0, 100 - (size_variance / avg_size) * 10)
    print(f"   청크 균일성: {uniformity_score:.1f}% ({'양호' if uniformity_score > 70 else '개선 필요'})")
    
    # RAG 검색 최적화 점수
    optimal_chunks = sum(1 for size in chunk_sizes if 500 <= size <= 1500)
    optimization_score = (optimal_chunks / len(chunks)) * 100
    print(f"   RAG 최적화 점수: {optimization_score:.1f}% ({optimal_chunks}/{len(chunks)} 청크가 최적 범위)")
    
    # 겹침 정보
    overlapped_chunks = sum(1 for chunk in chunks if chunk.get("overlap_with_next", False))
    print(f"   겹침 청크: {overlapped_chunks}개 ({(overlapped_chunks/len(chunks))*100:.1f}%)")
    
    return {
        "total_chunks": len(chunks),
        "avg_tokens": avg_size,
        "uniformity_score": uniformity_score,
        "optimization_score": optimization_score,
        "ready_for_embedding": optimization_score > 50
    }

# RAG 파이프라인 청킹 테스트
print("🎯 RAG 파이프라인 한국어 청킹 테스트")
print("=" * 60)

if 'result' in locals() and result.get("success"):
    print("📄 11번 셀 Textract 결과를 사용한 청킹")
    
    # RAG 메타데이터 활용
    rag_metadata = result.get("rag_metadata")
    if rag_metadata and rag_metadata.get("ready_for_chunking"):
        print(f"✅ 청킹 준비 완료 - 예상 청크 수: {rag_metadata['estimated_chunks']}개")
        
        # RAG 최적화 청킹 실행
        chunks = chunk_korean_text_for_rag(
            result["text"], 
            max_tokens=1200, 
            overlap_ratio=0.1,
            rag_metadata=rag_metadata
        )
        
        # 청크 품질 분석
        chunk_analysis = analyze_chunks_for_rag(chunks)
        
        # 샘플 청크 출력
        print(f"\n? 샘플 청크 미리보기:")
        for i, chunk in enumerate(chunks[:3]):
            print(f"\n🔸 청크 {i+1} (토큰: {chunk.get('tokens', '추정')}개):")
            print(f"   {chunk['text'][:150]}...")
            if len(chunk['text']) > 150:
                print("   ... (더 많은 내용)")
        
        # RAG 파이프라인 준비 상태
        if chunk_analysis and chunk_analysis["ready_for_embedding"]:
            print(f"\n✅ RAG 파이프라인 준비 완료!")
            print(f"   임베딩 대상: {chunk_analysis['total_chunks']}개 청크")
            print(f"   최적화 점수: {chunk_analysis['optimization_score']:.1f}%")
        else:
            print(f"\n⚠️ 청킹 최적화 필요")
            print(f"   청크 수가 적거나 크기가 불균일할 수 있습니다")
    else:
        print("⚠️ RAG 메타데이터가 없거나 청킹 준비가 되지 않았습니다")
        print("기본 청킹을 시도합니다...")
        
        # 기본 청킹
        chunks = chunk_korean_text_for_rag(result["text"])
        chunk_analysis = analyze_chunks_for_rag(chunks)
        
        # 샘플 출력
        if chunks:
            print(f"\n📖 기본 청킹 결과 미리보기:")
            for i, chunk in enumerate(chunks[:2]):
                print(f"\n🔸 청크 {i+1}: {chunk['text'][:100]}...")

elif 'result' in locals():
    print(f"❌ 11번 셀 실행 결과에 오류가 있습니다: {result.get('error', 'Unknown error')}")
    print("11번 셀을 다시 실행해 주세요.")
else:
    print("❌ 11번 셀을 먼저 실행하여 Textract 결과를 생성해 주세요.")

print(f"\n🏁 RAG 파이프라인 청킹 테스트 완료")

## 5. 백엔드 통합 테스트

In [ ]:
# 백엔드 인증
def login_to_backend(emp_no="HR001", password="hr123!"):
    """백엔드 시스템에 로그인"""
    if not BACKEND_AVAILABLE:
        print("❌ 백엔드 서비스를 사용할 수 없습니다")
        return None
    
    try:
        login_url = f"{BACKEND_URL}/api/v1/auth/login"
        response = requests.post(
            login_url,
            json={"emp_no": emp_no, "password": password},
            timeout=10
        )
        response.raise_for_status()
        
        data = response.json()
        token = data.get("access_token")
        
        if token:
            print(f"✅ 로그인 성공: {emp_no}")
            return token
        else:
            print("❌ 토큰을 받지 못했습니다")
            return None
            
    except Exception as e:
        print(f"❌ 로그인 실패: {e}")
        return None

# 로그인 테스트
access_token = login_to_backend()

In [ ]:
# 컨테이너 조회
def get_containers(token):
    """사용 가능한 지식 컨테이너 목록 조회"""
    if not token:
        print("❌ 인증 토큰이 필요합니다")
        return []
    
    try:
        headers = {"Authorization": f"Bearer {token}"}
        containers_url = f"{BACKEND_URL}/api/v1/documents/containers"
        
        response = requests.get(containers_url, headers=headers, timeout=10)
        response.raise_for_status()
        
        containers = response.json()
        print(f"📂 사용 가능한 컨테이너: {len(containers)}개")
        
        for container in containers[:3]:
            name = container.get('container_name', 'Unknown')
            desc = container.get('description', 'No description')
            print(f"  • {name}: {desc}")
        
        return containers
        
    except Exception as e:
        print(f"❌ 컨테이너 조회 실패: {e}")
        return []

# 컨테이너 조회 테스트
containers = get_containers(access_token)
selected_container = containers[0]['container_id'] if containers else None

In [ ]:
# 하이브리드 검색 테스트
def test_hybrid_search(token, query="문서 처리", limit=5):
    """백엔드 하이브리드 검색 테스트"""
    if not token:
        print("❌ 인증 토큰이 필요합니다")
        return []
    
    try:
        headers = {"Authorization": f"Bearer {token}"}
        search_url = f"{BACKEND_URL}/api/v1/search"
        
        search_data = {
            "query": query,
            "limit": limit,
            "search_type": "hybrid"
        }
        
        response = requests.post(
            search_url, 
            json=search_data, 
            headers=headers, 
            timeout=15
        )
        response.raise_for_status()
        
        results = response.json()
        documents = results.get('documents', [])
        
        print(f"🔍 검색 결과: '{query}' → {len(documents)}건")
        
        for i, doc in enumerate(documents[:3]):
            title = doc.get('title', 'Unknown')
            score = doc.get('score', 0)
            print(f"  {i+1}. {title} (점수: {score:.3f})")
        
        return documents
        
    except Exception as e:
        print(f"❌ 검색 실패: {e}")
        return []

# 검색 테스트
search_results = test_hybrid_search(access_token, "테스트 문서")

## 6. 성능 및 결과 요약

In [ ]:
# RAG 파이프라인 전체 성능 및 결과 요약
def print_rag_pipeline_summary():
    """RAG 파이프라인 Textract 통합 테스트 결과 요약"""
    print("\n" + "="*60)
    print("📊 RAG 파이프라인 Textract 통합 테스트 결과")
    print("="*60)
    
    # 환경 설정 상태
    print(f"🔧 백엔드 연결: {'✅ 성공' if BACKEND_AVAILABLE else '❌ 실패'}")
    print(f"☁️ AWS Textract: {'✅ 사용 가능' if textract_client else '❌ 사용 불가'}")
    print(f"🪣 S3 연동: {'✅ 설정됨' if TEXTRACT_S3_BUCKET else '❌ 미설정'}")
    print(f"🇰🇷 한국어 NLP: {'✅ 사용 가능' if kiwi and tokenizer else '❌ 사용 불가'}")
    
    # RAG 파이프라인 처리 결과
    if 'result' in locals() and result.get("success"):
        print(f"\n📄 RAG 파이프라인 문서 처리: ✅ 성공")
        print(f"   처리 방식: {result['api_used']}")
        print(f"   추출된 텍스트: {len(result['text'])} 문자")
        print(f"   블록 수: {result['blocks_count']}")
        
        # RAG 메타데이터 요약
        if 'rag_metadata' in result:
            meta = result['rag_metadata']
            print(f"\n📋 RAG 메타데이터:")
            print(f"   파일 크기: {meta['file_size']/1024/1024:.2f}MB")
            print(f"   문서 복잡도: {meta['complexity']}")
            print(f"   처리 시간: {meta['extracted_at']}")
            print(f"   콘텐츠 구성:")
            print(f"     - 텍스트 라인: {meta['content_types']['text_lines']}개")
            print(f"     - 테이블: {meta['content_types']['tables']}개")
            print(f"     - 폼 필드: {meta['content_types']['forms']}개")
            print(f"   청킹 준비: {'✅ 준비됨' if meta['ready_for_chunking'] else '❌ 준비 안됨'}")
            print(f"   예상 청크 수: {meta['estimated_chunks']}개")
        
        # 청킹 결과
        if 'chunks' in locals() and chunks:
            chunk_count = len(chunks)
            avg_tokens = sum(chunk.get("tokens", len(chunk["text"])//4) for chunk in chunks) / chunk_count
            print(f"\n✂️ 한국어 청킹 결과: ✅ 성공")
            print(f"   생성된 청크: {chunk_count}개")
            print(f"   평균 토큰 수: {avg_tokens:.1f}")
            
            # 청킹 품질 평가
            if 'chunk_analysis' in locals() and chunk_analysis:
                print(f"   청킹 품질:")
                print(f"     - 균일성: {chunk_analysis.get('uniformity_score', 0):.1f}%")
                print(f"     - RAG 최적화: {chunk_analysis.get('optimization_score', 0):.1f}%")
                print(f"     - 임베딩 준비: {'✅ 준비됨' if chunk_analysis.get('ready_for_embedding', False) else '⚠️ 최적화 필요'}")
        else:
            print(f"\n✂️ 한국어 청킹: ❌ 실패 또는 미실행")
    else:
        error_msg = result.get('error', 'Unknown error') if 'result' in locals() else '미실행'
        pipeline_stage = result.get('pipeline_stage', 'unknown') if 'result' in locals() else 'unknown'
        print(f"\n📄 RAG 파이프라인 문서 처리: ❌ 실패")
        print(f"   오류: {error_msg}")
        print(f"   실패 단계: {pipeline_stage}")
    
    # 백엔드 통합 상태
    if access_token:
        print(f"\n🔐 백엔드 통합: ✅ 성공")
        print(f"   인증 토큰: 활성")
        container_count = len(containers) if 'containers' in locals() and containers else 0
        search_count = len(search_results) if 'search_results' in locals() and search_results else 0
        print(f"   컨테이너 조회: {container_count}개")
        print(f"   검색 테스트: {search_count}건")
    else:
        print(f"\n🔐 백엔드 통합: ❌ 실패")
        print("   인증 토큰 없음")
    
    # RAG 파이프라인 준비도 평가
    print(f"\n🎯 RAG 파이프라인 준비도 평가:")
    
    # 기본 인프라
    infra_score = 0
    if textract_client: infra_score += 25
    if TEXTRACT_S3_BUCKET: infra_score += 25
    if kiwi and tokenizer: infra_score += 25
    if access_token: infra_score += 25
    
    print(f"   인프라 준비도: {infra_score}% ({infra_score//25}/4 구성요소)")
    
    # 처리 성능
    processing_score = 0
    if 'result' in locals() and result.get("success"):
        processing_score += 50
        if 'rag_metadata' in result and result['rag_metadata'].get('ready_for_chunking'):
            processing_score += 25
        if 'chunks' in locals() and chunks:
            processing_score += 25
    
    print(f"   처리 성능: {processing_score}% ({processing_score//25}/4 단계)")
    
    # 전체 준비도
    overall_readiness = (infra_score + processing_score) // 2
    print(f"   전체 준비도: {overall_readiness}% ", end="")
    
    if overall_readiness >= 80:
        print("🟢 운영 준비 완료")
        print("   💡 권장 사항: 성능 최적화 및 모니터링 구축")
    elif overall_readiness >= 60:
        print("🟡 부분 준비 완료")
        print("   💡 권장 사항: 미완성 구성요소 보완 필요")
    else:
        print("🔴 준비 부족")
        print("   💡 권장 사항: 기본 인프라 구성부터 재검토")
    
    # 다음 단계 제안
    print(f"\n🚀 RAG 파이프라인 다음 단계:")
    
    if overall_readiness >= 80:
        print("1. 📊 대용량 문서 배치 처리 테스트")
        print("2. 🔍 임베딩 벡터 생성 및 검색 성능 테스트")
        print("3. 📈 파이프라인 성능 모니터링 구축")
        print("4. 🛡️ 프로덕션 보안 설정 검토")
    elif overall_readiness >= 60:
        print("1. ❌ 실패한 구성요소 문제 해결")
        print("2. 🔧 청킹 파라미터 최적화")
        print("3. 📝 오류 처리 로직 강화")
        print("4. 🧪 추가 문서 형식 테스트")
    else:
        print("1. 🔧 AWS 권한 및 S3 설정 재확인")
        print("2. 🔐 백엔드 연결 문제 해결")
        print("3. 📚 한국어 NLP 라이브러리 설치")
        print("4. 🏗️ 기본 인프라 재구성")
    
    print("="*60)
    print("✨ RAG 파이프라인 Textract 통합 테스트 완료")
    
    return {
        "infra_score": infra_score,
        "processing_score": processing_score,
        "overall_readiness": overall_readiness
    }

# RAG 파이프라인 요약 실행
pipeline_summary = print_rag_pipeline_summary()

# 추가 테스트 권장사항
if 'result' in locals() and result.get("success") and 'chunks' in locals():
    print(f"\n💡 추가 테스트 파일 권장사항:")
    
    current_size = result.get('rag_metadata', {}).get('file_size', 0)
    if current_size < 1024 * 1024:  # 1MB 미만
        print("📄 소형 문서 테스트 완료 - 중형/대형 문서로 추가 테스트 권장")
        print("   권장 테스트: 5MB-50MB PDF, 복잡한 표가 포함된 문서")
    elif current_size < 10 * 1024 * 1024:  # 10MB 미만
        print("📄 중형 문서 테스트 완료 - 대형 문서로 추가 테스트 권장") 
        print("   권장 테스트: 50MB+ PDF, 다중 페이지 스캔 문서")
    else:
        print("📄 대형 문서 테스트 완료 - 비동기 처리 최적화 검토 권장")
        print("   권장 테스트: 다양한 문서 형식 (PPT, Word, 이미지)")
    
    print("🔍 성능 개선 영역:")
    if 'chunk_analysis' in locals():
        if chunk_analysis.get('optimization_score', 0) < 70:
            print("   - 청킹 파라미터 최적화 (토큰 크기, 겹침 비율)")
        if chunk_analysis.get('uniformity_score', 0) < 70:
            print("   - 청크 크기 균일성 개선")
    
    confidence_avg = 85  # 예시 값 (실제로는 Textract 응답에서 계산)
    if confidence_avg < 90:
        print("   - OCR 품질 개선 (이미지 전처리, 해상도 최적화)")
    
    print("\n📋 테스트 결과 기록:")
    print(f"   테스트 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"   테스트 파일: {test_file if 'test_file' in locals() else 'Unknown'}")
    print(f"   처리 성공률: {'100%' if result.get('success') else '0%'}")
    print(f"   RAG 파이프라인 준비도: {pipeline_summary['overall_readiness']}%")

## 다음 단계

파이프라인 테스트가 완료되었습니다. 다음과 같은 작업을 진행할 수 있습니다:

1. **문서 업로드**: 실제 문서를 백엔드에 업로드하여 전체 처리 과정 테스트
2. **성능 튜닝**: 청킹 파라미터 조정 및 임베딩 최적화
3. **품질 평가**: 추출된 텍스트의 정확도 및 검색 성능 평가
4. **확장 기능**: 표/이미지 추출, 멀티모달 처리 등 고급 기능 테스트

각 단계에서 발생한 문제점이 있다면 해당 셀을 다시 실행하거나 설정을 확인해 주세요.

## 한국어 OCR 대안 분석 및 구현

AWS Textract는 한국어 지원이 제한적입니다. 한국어 문서에 더 적합한 OCR 대안들을 평가해보겠습니다:

In [ ]:
# 빠른 테스트 - 첫 2페이지만 처리
test_file = '/home/admin/wkms-aws/jupyter_notebook/data/input_docs/test.pdf'

# 페이지별 레이아웃 설정
layout_settings = {
    'enable_page_specific': ENABLE_PAGE_SPECIFIC_LAYOUT,
    'first_page_layout': FIRST_PAGE_LAYOUT
}

print("🔥 빠른 테스트 시작 - 첫 2페이지만 처리")
print(f"📋 레이아웃 설정: {layout_settings}")

# 빠른 미리보기로 첫 2페이지만 추출
result = extract_text_fast_preview(test_file, page_limit=2, layout_settings=layout_settings)

if result:
    print(f"\n✅ 추출 완료: {len(result['full_text'])}자")
    print(f"📄 페이지 수: {result['total_pages']}")
    
    # 페이지별 결과 요약
    for page_info in result['page_results']:
        print(f"   페이지 {page_info['page']}: {page_info['layout_type']}, {page_info['lines_count']}라인, {len(page_info['text'])}자")
    
    # 첫 300자 출력
    print(f"\n📝 첫 300자 미리보기:")
    print(result['full_text'][:300])
    
    # 파일 저장
    output_dir = "/home/admin/wkms-aws/jupyter_notebook/data/output_texts"
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    quick_filename = f"test_quick_{timestamp}.txt"
    with open(os.path.join(output_dir, quick_filename), 'w', encoding='utf-8') as f:
        f.write(result['full_text'])
    print(f"\n💾 저장완료: {quick_filename}")
else:
    print("❌ 추출 실패")